# 작사가 인공지능 만들기

▶목차


|   Chapter               | details             |
|:-------------------------|:---------------------|
|4-7 |프로젝트: 작사가 인공지능 만들기         |
|Step 1 | 데이터 다운로드 |
|Step 2 | 데이터 읽어오기       |
|Step 3 | 데이터 정제               |
|Step 4 |평가 데이터셋 분리                  |
|Step 5 |인공지능 만들기                  |
|회고 | 느낀점 & 다양한 시도     |
|Final |깃허브 주소 업로드                           |

▶딥러닝 순서


  | 딥러닝순서                                             | 상세내용                                       |
|:------------------------------------------------------|:------------------------------------------------|
|1. 데이터 준비       |데이터셋을 구성한다                |
|2. 딥러닝 네트워크 설계   |데이터셋의 다양성, 정규화 등의 시도가 적절하였음|
|3. 학습|학습모델을 사용하여 학습시킨다.                             |
|4. 테스트(평가)| 학습한 내용을 파악하고 예측을 시도해본다|

▶루브릭 평가 기준

    
- 아래의 기준을 바탕으로 프로젝트를 평가합니다.

| 평가문항                                             | 상세기준                                       |
|:------------------------------------------------------|:------------------------------------------------|
|1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?   |텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?                |
|2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?   |특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?|
|3. 텍스트 생성모델이 안정적으로 학습되었는가?|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?|


## 1. 데이터 다운로드

### 폴더 생성 후 데이터 이동

$ mkdir -p ~/aiffel/lyricist/models
$ ln -s ~/data ~/aiffel/lyricist/data

## 2. 데이터 읽어오기

In [3]:
import glob
import os, re
import numpy as np
import tensorflow as tf


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10]) # 앞에서부터 10라인만 화면에 출력

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof']


# 3. 데이터 정제

## 3-1. 정규표현식을 이용한 corpus 생성
- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환
- 정제함수 = Corpus 사용법
- Corpus = [ ], ex) Corpus [:10], Corpus.append() 

In [7]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("The purification function @_is ;;; is corpus."))
print(preprocess_sentence(raw_corpus[0]))
print(preprocess_sentence(raw_corpus[1]))
print(preprocess_sentence(raw_corpus[2]))
print(preprocess_sentence(raw_corpus[3]))
print(preprocess_sentence(raw_corpus[4]))
print(preprocess_sentence(raw_corpus[5]))
print(preprocess_sentence(raw_corpus[6]))
print(preprocess_sentence(raw_corpus[7]))

<start> the purification function is is corpus . <end>
<start> now i ve heard there was a secret chord <end>
<start> that david played , and it pleased the lord <end>
<start> but you don t really care for music , do you ? <end>
<start> it goes like this <end>
<start> the fourth , the fifth <end>
<start> the minor fall , the major lift <end>
<start> the baffled king composing hallelujah hallelujah <end>
<start> hallelujah <end>


In [9]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0 or len(sentence.split()) > 15: continue   # 길이가 0이거나 단어가 15개 넘는 긴 문장은 제외
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[10:20]

['<start> you saw her bathing on the roof <end>',
 '<start> her beauty and the moonlight overthrew her <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> she broke your throne , and she cut your hair <end>',
 '<start> and from your lips she drew the hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah you say i took the name in vain <end>',
 '<start> i don t even know the name <end>']

## Tokenizer 패키지로 corpus를 텐서로 변환
### 단어장의 크기는 12,000 이상 으로 설정하세요 

In [12]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12,000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12,000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2706 ...    0    0    0]
 [   2   34    7 ...   44    3    0]
 ...
 [   2  259  194 ...   12    3    0]
 [   5   22    9 ...   10 1099    3]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff5a17fea90>


In [19]:
# 변환된 텐서 확인
print(tensor[:6, :20] )
tensor.shape

[[   2   50    5   91  307   62   57    9  957 5739    3    0    0    0
     0]
 [   2   17 2706  879    4    8   11 6171    6  347    3    0    0    0
     0]
 [   2   34    7   35   15  162  283   28  334    4   48    7   44    3
     0]
 [   2   11  346   24   42    3    0    0    0    0    0    0    0    0
     0]
 [   2    6 4332    4    6 2172    3    0    0    0    0    0    0    0
     0]
 [   2    6 5740  292    4    6 1211  790    3    0    0    0    0    0
     0]]


(168357, 15)

### tokenizer에 구축된 단어 사전의 인덱스입니다. 단어 사전이 어떻게 구축되었는지 아래와 같이 확인해 봅시다.

In [20]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


2번 인덱스가 바로 <start>였습니다.
왜 모든 행이 2로 시작하는지 이해할 수 있겠습니다.

In [21]:
# source(소스문장), target(정답문장) 생성
src_input = tensor[:, :-1]   # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])
src_input.shape

[   2   50    5   91  307   62   57    9  957 5739    3    0    0    0]
[  50    5   91  307   62   57    9  957 5739    3    0    0    0    0]


(168357, 14)

# 4. 평가 데이터셋 분리

In [22]:
from sklearn.model_selection import train_test_split

# tensor를 train, test 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (134685, 14)
Target Train: (134685, 14)


# 5. 모델 설계 및 훈련

In [23]:
vocab_size = tokenizer.num_words + 1  # 단어사전의 단어 개수 + 0:<pad>
embedding_size = 256
hidden_size = 1024

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_size))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.Dense(vocab_size))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         3072256   
_________________________________________________________________
lstm (LSTM)                  (None, None, 1024)        5246976   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
dense (Dense)                (None, None, 12001)       12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [24]:
# optimization 최적화, loss 손실, Epochs 반복학습 횟수 지정
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
4209/4209 [==============================] - 139s 31ms/step - loss: 3.2191
Epoch 2/10
4209/4209 [==============================] - 134s 32ms/step - loss: 2.7693
Epoch 3/10
4209/4209 [==============================] - 134s 32ms/step - loss: 2.4927
Epoch 4/10
4209/4209 [==============================] - 134s 32ms/step - loss: 2.2462
Epoch 5/10
4209/4209 [==============================] - 134s 32ms/step - loss: 2.0301
Epoch 6/10
4209/4209 [==============================] - 134s 32ms/step - loss: 1.8426
Epoch 7/10
4209/4209 [==============================] - 134s 32ms/step - loss: 1.6799
Epoch 8/10
4209/4209 [==============================] - 133s 32ms/step - loss: 1.5399
Epoch 9/10
4209/4209 [==============================] - 133s 32ms/step - loss: 1.4206
Epoch 10/10
4209/4209 [==============================] - 133s 32ms/step - loss: 1.3194


# 5. 모델 평가

In [25]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

# 언어모델 테스트

In [30]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love you , liberian girl <end> '

In [31]:
generate_text(model, tokenizer, init_sentence="<start> i need", max_len=15)

'<start> i need a girl from <unk> <end> '

In [32]:
generate_text(model, tokenizer, init_sentence="<start> i wanna", max_len=15)

'<start> i wanna be with you , i wanna be with you <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> the like", max_len=15)

'<start> the like that i could claim <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> you like", max_len=15)

'<start> you like sleeping alone <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> and you", max_len=15)

'<start> and you re the one <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> to you", max_len=15)

'<start> to you i m not <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> i lucky", max_len=15)

'<start> i lucky that you re the only one <end> '

# 회고
> 이게 RNN(순환신경망)인가 싶은데요
아직도 갈 길은 멀고 이해를 바로 하기에는 많은 부분이 어렵네요.
뭔가 할 듯 하면서도 모르는 것 같고 모르는 것 같으면서도 알 것 같은 이 묘한 상황
어디서 부터 어디까지 배워야 하는 건지 감을 못 잡고 있어서 배워야 할 것은 참 많다라는 생각이 듭니다.
반복학습 에포크는 너무 오래 걸림을 다시한번 더 느꼈습니다.

# 마무리
- 단어장의 크기는 12,000 으로 설정
- 길이가 0이거나 단어가 15개 넘는 긴 문장은 제외
- Epoch 10/10 
- loss: 1.3194
loss 값은 1.31로 낮게 나왔다.


#### reference
https://github.com/kec0130/AIFFEL-project/blob/main/exploration/E6_lyricist.ipynb